# Generating

In [61]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

print(f"Using device: {device}")

Using device: mps


In [2]:
from torchvision.datasets import ImageFolder

test_dir = "./data/test"
test_ds = ImageFolder(test_dir)
len(test_ds), len(test_ds.classes)

FileNotFoundError: Couldn't find any class folder in ./data/test.

## cVAE

## Generating tensor and images

In [66]:
total_test_samples = sum(class_counts.values())
class_ratios = {cls: count / total_test_samples for cls, count in class_counts.items()}

# ile próbek z każdej klasy w puli 1000
samples_per_class = {cls: int(round(ratio * num_samples)) for cls, ratio in class_ratios.items()}
samples_per_class.keys(), samples_per_class.values()


(dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]),
 dict_values([5, 57, 57, 36, 50, 47, 11, 37, 36, 37, 51, 34, 54, 55, 20, 16, 11, 28, 31, 5, 9, 8, 10, 13, 7, 38, 15, 6, 14, 7, 11, 20, 6, 18, 11, 31, 10, 5, 53, 8, 9, 6, 6]))

In [ ]:
from gnn.models import Generator, Discriminator
import torch.optim as optim
import torch.nn as nn


latent_dim = 128
num_classes = len(test_ds.classes)

# model

generator = Generator(latent_dim, num_classes).to(device)
generator.load_state_dict(torch.load("1747144428.030544/generator.pth"))
generator.eval()

discriminator = Discriminator(latent_dim, num_classes).to(device)
discriminator.load_state_dict(torch.load("1747144428.030544/discriminator.pth"))
discriminator.eval()

In [71]:
import os
import time
from torchvision.utils import save_image
from collections import Counter


results_id = time.time()


results_dir_jpg = f"./cvae_results/jpg/{results_id}"
results_dir_pt = f"./cvae_results/pt/{results_id}"

os.makedirs(results_dir_jpg, exist_ok=True)
os.makedirs(results_dir_pt, exist_ok=True)

train_mean = [0.5, 0.5, 0.5]
train_std = [0.5, 0.5, 0.5]


mean_t = torch.tensor(train_mean).view(1, IMG_CHANNELS, 1, 1).to(device)
std_t = torch.tensor(train_std).view(1, IMG_CHANNELS, 1, 1).to(device)
class_counts = Counter(test_ds.targets)


num_samples = 1000
total_test_samples = sum(class_counts.values())
class_ratios = {cls: count / total_test_samples for cls, count in class_counts.items()}
samples_per_class = {cls: int(round(ratio * num_samples)) for cls, ratio in class_ratios.items()}

print(f"Init samples: {sum(samples_per_class.values())}")
# adjust to 1000
adjustment = num_samples - sum(samples_per_class.values())
if adjustment != 0:
    most_common_class = max(samples_per_class, key=samples_per_class.get)
    samples_per_class[most_common_class] += adjustment
print(f"Adjusted samples: {sum(samples_per_class.values())}")


def _save(model):
    generated_imgs = []
    for cls, count in samples_per_class.items():
        for i in range(count):
            z = torch.randn(1, latent_dim, device=device)
            label_tensor = torch.tensor([cls], dtype=torch.long, device=device)

            with torch.no_grad():
                img = model.generate(z, label_tensor)

            img = img * std_t + mean_t
            generated_imgs.append(img.cpu().detach())

            # Save the image
            fname = os.path.join(results_dir_jpg, f"class_{cls}_sample_{i}.jpg")
            save_image(img.clamp(0, 1), fname)

    print(f"Saved generated images to {results_dir_jpg}")
    # Save the tensor
    generated_imgs = torch.cat(generated_imgs, dim=0)
    return generated_imgs


Init samples: 999
Adjusted samples: 1000


In [72]:
# Evaluating
print("Evaluating using model name: ", cvae_model.__class__.__name__)
# cvae_model.eval()

generated_imgs = _save(cvae_model)

# save tensor
assert generated_imgs.shape == (1000, 3, 32, 32), f"Zły rozmiar tensora: {generated_imgs.shape}"
fname = os.path.join(results_dir_pt, f"poniedzialek_matukiewicz_statkiewicz.pt")                # TODO nazwiska i dzien sprawdzic ! ! !
torch.save(generated_imgs, fname)

print(f"Saved generated tensor to {results_dir_pt}")
 

Evaluating using model name:  BigConditionalVariationalAutoencoder
Saved generated images to ./cvae_results/jpg/1747141943.95138
Saved generated tensor to ./cvae_results/pt/1747141943.95138


## FID

### Fid ConditionalVariationalAutoencoder

In [69]:
from pytorch_fid.fid_score import calculate_fid_given_paths

test_flat_dir = "./data/test_flat"
generated_dir = results_dir_jpg

batch_size = 128

fid = calculate_fid_given_paths([test_flat_dir, generated_dir], batch_size, device, dims=2048, num_workers=1)

print(f"FID: {fid}")

100%|██████████| 8/8 [00:05<00:00,  1.51it/s]


FID: 130.60183958428928


#### Fid BigConditionalVariationalAutoencoder

In [73]:
from pytorch_fid.fid_score import calculate_fid_given_paths

test_flat_dir = "./data/test_flat"
generated_dir = results_dir_jpg

batch_size = 128

fid = calculate_fid_given_paths([test_flat_dir, generated_dir], batch_size, device, dims=2048, num_workers=1)

print(f"FID: {fid}")

100%|██████████| 8/8 [00:05<00:00,  1.45it/s]


FID: 127.72886070703294
